In [15]:
from scipy.stats import chi2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import urllib

# Read a .txt or .csv file containing SNPs and P values to be plotted
# Function checks delimiters automatically
# Parameter: filename - .txt or .csv file containing information needed
# Return: pandas dataframe of the file
def __read_file(filename):
    fh = open(filename, 'r')
    line = fh.readline()
    tmp = line.strip().split(',')

    # Create an empty dataframe, read data into df later
    df = pd.DataFrame()
    # Check delimiter type, "," or " "
    if len(tmp) == 1:
        df = pd.read_csv(filename, delim_whitespace=True)
    else:
        df = pd.read_csv(filename)
    
    return(df)

# Return -log10 transformed observed and expected p vlaues of a dataset(dataframe)
def __log_obsv_and_expt(df, column_title):
    Pobsv = df.loc[:, column_title]  # Observed p values
    Pobsv = np.sort(Pobsv.dropna().values)
    Pexpt = np.arange(1, len(Pobsv) + 1, 1) / (len(Pobsv) + 1)  # Expected
    logPobsv = -np.log10(Pobsv)
    logPexpt = -np.log10(Pexpt)
    
    return(Pobsv, logPobsv, logPexpt)


# Calculate and return lambda (inflation)
def __get_inflation__(observed):
    obsv_median = np.median(observed)
    Chi = chi2.ppf(1.0 - obsv_median, 1)
    lmbd = Chi / chi2.ppf(0.5, 1)
    return lmbd


# The tricky part for me is to query haploreg database thorugh their website.
# Inspecct the element of the website to find out names of each variable
# Recommand using Chrome, go to developer mode, refresh the website under Network tab
# This function is intended to query one SNP at a time
# Return: a set of rsIDs of accociated SNPs (and the queried SNP)

# Parameters needed to send to haploreg website, and some explaination
# input_snps (query): input SNPs in a string, each SNP is separated by ','
#        SNP number per query is set to be 1000, otherwise it takes too long and haploreg may refuse to do it.
#        15000 SNPs did not work for test run.
# gwas_id: the dropdown list to choose GWAS study, when no file or query SNP(s) is provided
# r2_threshold (ldThresh): r^2 threshold, default is 0.2 in this code
# ldPop: 1000G Phase 1 population for LD calculation
# epi: Source for epigenomes
# cons: Mammalian conservation algorithm. 'siphy'=SiPhy-omega, 'gerp'=GERP, or 'both'
# genetypes: Show position relative to
# output: set output result type to 'text' for python code to process
def single_query_haploreg(input_snp,
                   r2_threshold=0.2,
                   ldPop='EUR',
                   epi='vanilla',
                   cons='siphy',
                   genetypes='gencode'):

    params_library = {'query':input_snp,
                      'gwas_id':0,
                      'ldThresh': r2_threshold,
                      'ldPop': ldPop,
                      'epi': epi,
                      'cons': cons,
                      'genetypes': genetypes,
                      'output':'text'}
    
    # parameters passed to the website, needs to be parsed and in binary
    params = urllib.parse.urlencode(params_library).encode("utf-8")
    # url of HaploReg4.1
    url = 'https://pubs.broadinstitute.org/mammals/haploreg/haploreg.php'
    # Query with parameters
    query = urllib.request.urlopen(url, params)
    content = query.read().decode("utf-8")
    # Find accociated SNPs in the content returned from HaploReg
    matches = re.findall('rs[0-9]+', content)

    # Return unique ones of the query
    return(set(matches))
#     return content


# Queried SNPs are based on the file provided
# Parameter:
# - 
# -
# Return: Save novel SNPs as .csv file fot plotting and reference
def __novel_SNPs(filename, r2_threshold, SNPs_column_title, novel_filename='novel_SNPs.csv'):
    original_df = __read_file(filename) # Dataframe read from the file
    all_SNPs = original_df.loc[:, SNPs_column_title].values # Get all SNPs in the file to be queried
    associated_SNPs = np.array([]) # A numpy array to store queried results
    
    # Count is to show numbers of SNPs processed so far, to keep the console updating
    count = 0
    for snp in all_SNPs:
        associated_SNPs = np.append(associated_SNPs, single_query_haploreg(input_snp=snp, r2_threshold=r2_threshold))
        
        count = count+1
        # Print every 10 SNPs
        if count%10 == 0:
            print('Number of SNPs processed: ' + str(count))
        
    
    # Remove duplicates of queried results
    associated_SNPs = np.unique(associated_SNPs)
    
    # Join the original dataframe and queried results, output novel SNPs
    original_df.set_index(SNPs_column_title, inplace=True)
    associated_df = pd.DataFrame([], index=associated_SNPs)
    remove_df = original_df.join(associated_df, how='inner')
    novel_df = original_df.drop(remove_df.index.values)
    
    novel_df.to_csv(novel_filename)

# Returns: (fig, ax, lambda)
# fig and ax for more custermizations
def qqplot(filename,
           output='output.png',
           column_title = 'P',
           title='Q-Q plot',
           xlabel='Expected –log10 P-values',
           ylabel='Observed –log10 P-values',
           dpi=300,
           plot_novel = False,
           novel_filename = 'novel_SNPs.csv',
           SNPs_column_title = 'SNP',
           r2_threshold=0.2):
    
    df = __read_file(filename)
    
#     Pobsv = df.loc[:, column_title]  # Observed p values
#     Pobsv = np.sort(Pobsv.dropna().values)
#     Pexpt = np.arange(1, len(Pobsv) + 1, 1) / (len(Pobsv) + 1)  # Expected
#     logPobsv = -np.log10(Pobsv)
#     logPexpt = -np.log10(Pexpt)

    Pobsv, logPobsv, logPexpt = __log_obsv_and_expt(df, column_title)

    # Calculate lambda
    infl = __get_inflation__(Pobsv)

    fig, ax = plt.subplots(dpi=dpi)
    ax.plot(logPexpt, logPobsv, linestyle='', marker='o', markersize=3, markeredgewidth=0.5,
            fillstyle='none', color='k', alpha=0.8)
    ax.plot(logPexpt, logPexpt, color='r', linewidth=0.4)

    # Label x and y axis
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    annotation = "λ = " + str("{0:.4f}".format(infl))
    ax.annotate(annotation, xy=(0.7, 0.2), xycoords='axes fraction')
    
    if plot_novel == True:
        # Plot novel SNPs, then same steps as before
        __novel_SNPs(filename, r2_threshold, SNPs_column_title, novel_filename)
        novel_SNPs_df = __read_file(novel_filename)
        
#         Pobsv_novel = novel_SNPs_df.loc[:, column_title]  # Observed p values
#         Pobsv_novel = np.sort(Pobsv_novel.dropna().values)
#         Pexpt_novel = np.arange(1, len(Pobsv_novel) + 1, 1) / (len(Pobsv_novel) + 1)  # Expected
#         logPobsv_novel = -np.log10(Pobsv_novel)
#         logPexpt_novel = -np.log10(Pexpt_novel)
        Pobsv_novel, logPobsv_novel, logPexpt_novel = __log_obsv_and_expt(novel_SNPs_df, column_title)

        ax.plot(logPexpt_novel, logPobsv_novel, linestyle='', marker='o', markersize=3, markeredgewidth=0.5,
                fillstyle='none', color='g', alpha=0.8)
    
    fig.savefig(output)
    
    return (fig, ax, infl)  # Return for more custermizations



In [ ]:
# Takes a df and scan for novel SNPs
# Better to process about 100 SNPs at a time, so the process will not take too long.
# column_title is the titile of the column that contains rsIDs
# Output the reuslt into a .csv file
def __associated_SNPs(file_df, column_title='SNP', r2_threshold=0.2, output='associated_SNPs.txt'):
    all_SNPs = file_df.loc[:,column_title].values # Get all SNPs in the file to be queried
    associated_SNPs = set() # A numpy array to store queried results

    for snp in all_SNPs:
        associated_SNPs = np.append(associated_SNPs, single_query_haploreg(input_snp=snp, r2_threshold=r2_threshold))
    
    # Remove duplicates of queried results
    associated_SNPs = np.unique(associated_SNPs)
    np.savetxt(output, associated_SNPs, fmt='%s')

In [ ]:
# This can be single or multiple SNP queris (each rsID is separated by ',')
def query_haploreg(input_snp,
                   r2_threshold=0.2,
                   ldPop='EUR',
                   epi='vanilla',
                   cons='siphy',
                   genetypes='gencode'):

    params_library = {'query':input_snp,
                      'gwas_id':0,
                      'ldThresh': r2_threshold,
                      'ldPop': ldPop,
                      'epi': epi,
                      'cons': cons,
                      'genetypes': genetypes,
                      'output':'text'}
    
    # parameters passed to the website, needs to be parsed and in binary
    params = urllib.parse.urlencode(params_library).encode("utf-8")
    # url of HaploReg4.1
    url = 'https://pubs.broadinstitute.org/mammals/haploreg/haploreg.php'
    # Query with parameters
    query = urllib.request.urlopen(url, params)
    content = query.read().decode("utf-8")
    # Find accociated SNPs in the content returned from HaploReg
    matches = re.findall('rs[0-9]+', content)

    # Return unique ones of the query
    return(list(set(matches)))

# This function returns known SNPs (and associated SNPs) based on user input file
# Parameters:
#  - filename: a file contains known SNPs, provided by user
#  - r2_threshold: LD threshold, defualt is 0.2
#  - SNPs_column_title: Column title of SNPs in that file
# Return: a list of associated SNPs pulled from HaploReg and known SNPs provided by user
def __known_SNPs(filename, r2_threshold=0.2, SNPs_column_title='SNP'):
    known_SNPs_df = __read_file(filename) # Dataframe of known SNPs
    # Get all SNPs to be queried, join them by ',' and query together
    known_SNPs = ','.join(known_SNPs_df.loc[:, SNPs_column_title].values)
    # A numpy array to store queried results
    associated_and_known_SNPs = query_haploreg(input_snp=known_SNPs, r2_threshold=r2_threshold)
    
#     if len(known_SNPs_df)>100:


    # Remove duplicates of queried results and return
    return(set(associated_and_known_SNPs))

# This function removes known SNPs and save novel SNPs in a .csv file
# Parameters:
#  - original_df: a dataframe containing raw data of SNPs and p values
#  - known_SNPs: a list of SNPs found in other GWAS, that are associated with traits of interest
#  - SNPs_column_title: Column title of SNPs in that file
# Return: a datafram of novel SNPs and p values
def __remove_known_SNPs(original_df, known_SNPs, SNPs_column_title='SNP', novel_filename='novel_SNPs'):
    # Join the original dataframe and konw SNPs, output novel SNPs to be plotted
    original_df.set_index(SNPs_column_title, inplace=True)
    known_SNPs_df = pd.DataFrame([], index=known_SNPs)
    to_be_removed_SNPs_df = original_df.join(known_SNPs_df, how='inner')
    novel_df = original_df.drop(to_be_removed_SNPs_df.index.values)
    
    novel_df.to_csv(novel_filename)
    return(novel_df)
    

In [ ]:
filename = 'Data/DLD_GWAS_10.csv'
a = __known_SNPs(filename)

In [ ]:
original_df = __read_file('Data/ADD_only_DLD_GWAS.txt')
new_df = __remove_known_SNPs(original_df, a)

In [1]:
import pandas as pd
gwas_filename_01 = "/data100t1/share/DIAMANTE/mrmega_hdl_final_0619_rsids_locuszoom.result"
gwas_filename_02 = "/data100t1/share/DIAMANTE/mrmega_hdl_final_0619_rsids.result"

gwas_df = pd.read_csv(gwas_filename_02, sep='\t')

# fh = open(gwas_filename_02, 'r')
# count=0
# for line in fh:
#     print(line.strip().split('\t'))
#     count=count+1
#     if count>10: break

In [2]:
gwas_df.head()

,MarkerName,Chromosome,Position,EA,NEA,EAF,Nsample,Ncohort,Effects,beta_0,...,ndf_association,P-value_association,chisq_ancestry_het,ndf_ancestry_het,P-value_ancestry_het,chisq_residual_het,ndf_residual_het,P-value_residual_het,lnBF,Comments
0,rs147324274,10,100000012,G,A,0.996606,26037,4,-?????????+??????++,-1.252750,...,2.0,0.317464,0.677718,1.0,0.410374,0.223172,2.0,0.894415,-0.238903,NaN
1,rs144804129,10,100000122,T,A,0.997312,27449,6,-+?????+??+??????+-,0.086105,...,2.0,0.765692,0.000672,1.0,0.979314,3.196730,4.0,0.525459,-1.524780,NaN
2,rs6602381,10,10000018,A,G,0.541823,38176,19,+++--+-++-+-+--++++,0.029652,...,2.0,0.295628,0.875949,1.0,0.349314,12.777500,17.0,0.750949,-1.725790,NaN
3,rs189891329,10,10000033,G,A,0.994151,26037,4,-?????????+??????+-,-1.370290,...,2.0,0.475657,1.028940,1.0,0.310409,7.119850,2.0,0.028441,-0.643236,NaN
4,rs112832083,10,100000588,T,C,0.992032,28518,7,+??-?+?+??+??????+-,-0.237271,...,2.0,0.093044,1.776260,1.0,0.182610,2.751990,5.0,0.738157,0.428771,NaN


In [11]:
len(gwas_df)

33512268

In [3]:
len(gwas_df)

33783872

In [ ]:
qqplot(filename=gwas_filename_01,output='20190930_mrmega_hdl.png',
       column_title = 'P-value_association',title='mrmega_hdl')

(<Figure size 1800x1200 with 1 Axes>,
 1.0720526881638353)

In [4]:
novel_01 = '/data100t1/share/DIAMANTE/GWAS_catalog_results_HDL.txt'
novel_df = pd.read_csv(novel_01, sep='\t')

In [5]:
novel_df.head()

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
0,6/21/17,28270201,Nagy R,3/7/17,Genome Med,www.ncbi.nlm.nih.gov/pubmed/28270201,Exploration of haplotype research consortium i...,HDL cholesterol,"19,223 British ancestry individuals from 6863 ...",NaN,...,7.522879,NaN,0.867736,[0.56-1.18] mg/dl increase,Illumina [24111857] (imputed),N,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,GCST004207,Genome-wide genotyping array
1,2/14/19,29507422,Hoffmann TJ,3/5/18,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/29507422,A large electronic-health-record-based genome-...,High density lipoprotein cholesterol levels,"76,627 European ancestry individuals, 7,795 Hi...",NaN,...,13.698970,(EA),0.113000,unit increase,Affymetrix [at least 7091467] (imputed),N,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,GCST007140,Genome-wide genotyping array
2,2/14/19,29507422,Hoffmann TJ,3/5/18,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/29507422,A large electronic-health-record-based genome-...,High density lipoprotein cholesterol levels,"76,627 European ancestry individuals, 7,795 Hi...",NaN,...,13.698970,NaN,0.114000,unit increase,Affymetrix [at least 7091467] (imputed),N,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,GCST007140,Genome-wide genotyping array
3,5/12/14,24097068,Willer CJ,10/6/13,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/24097068,Discovery and refinement of loci associated wi...,HDL cholesterol,"94,595 European ancestry individuals","93,982 European ancestry individuals",...,15.000000,NaN,0.051000,[NR] unit decrease,NR [NR] (imputed),N,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,GCST002223,Genome-wide genotyping array
4,6/26/17,28334899,Spracklen CN,2/21/17,Hum Mol Genet,www.ncbi.nlm.nih.gov/pubmed/28334899,Association analyses of East Asian individuals...,HDL cholesterol levels,"34,930 East Asian ancestry individuals, 187,16...","8,741 Chinese ancestry individuals",...,15.045757,(Trans-ethnic initial),0.050600,[0.038-0.063] unit decrease (EA Beta values),"Affymetrix, Illumina [~ 1900000] (imputed)",N,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,GCST004232,Genome-wide genotyping array


In [12]:
len(novel_df.T)

38

In [13]:
novel_df.T

,0,1,2,3,4,5,6,7,8,9,...,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299
DATE ADDED TO CATALOG,6/21/17,2/14/19,2/14/19,5/12/14,6/26/17,2/14/19,2/14/19,2/14/19,2/14/19,11/26/18,...,7/28/18,2/14/19,2/14/19,11/26/18,2/14/19,2/14/19,11/26/18,11/26/18,2/14/19,2/14/19
PUBMEDID,28270201,29507422,29507422,24097068,28334899,29507422,29507422,29507422,29507422,30275531,...,29403010,29507422,29507422,30275531,29507422,29507422,30275531,30275531,29507422,29507422
FIRST AUTHOR,Nagy R,Hoffmann TJ,Hoffmann TJ,Willer CJ,Spracklen CN,Hoffmann TJ,Hoffmann TJ,Hoffmann TJ,Hoffmann TJ,Klarin D,...,Kanai M,Hoffmann TJ,Hoffmann TJ,Klarin D,Hoffmann TJ,Hoffmann TJ,Klarin D,Klarin D,Hoffmann TJ,Hoffmann TJ
DATE,3/7/17,3/5/18,3/5/18,10/6/13,2/21/17,3/5/18,3/5/18,3/5/18,3/5/18,10/1/18,...,2/5/18,3/5/18,3/5/18,10/1/18,3/5/18,3/5/18,10/1/18,10/1/18,3/5/18,3/5/18
JOURNAL,Genome Med,Nat Genet,Nat Genet,Nat Genet,Hum Mol Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,...,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet,Nat Genet
LINK,www.ncbi.nlm.nih.gov/pubmed/28270201,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/24097068,www.ncbi.nlm.nih.gov/pubmed/28334899,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/30275531,...,www.ncbi.nlm.nih.gov/pubmed/29403010,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/30275531,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/30275531,www.ncbi.nlm.nih.gov/pubmed/30275531,www.ncbi.nlm.nih.gov/pubmed/29507422,www.ncbi.nlm.nih.gov/pubmed/29507422
STUDY,Exploration of haplotype research consortium i...,A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...,Discovery and refinement of loci associated wi...,Association analyses of East Asian individuals...,A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...,"Genetics of blood lipids among ~300,000 multi-...",...,Genetic analysis of quantitative traits in the...,A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...,"Genetics of blood lipids among ~300,000 multi-...",A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...,"Genetics of blood lipids among ~300,000 multi-...","Genetics of blood lipids among ~300,000 multi-...",A large electronic-health-record-based genome-...,A large electronic-health-record-based genome-...
DISEASE/TRAIT,HDL cholesterol,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,HDL cholesterol,HDL cholesterol levels,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,HDL cholesterol,...,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,HDL cholesterol,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels,HDL cholesterol,HDL cholesterol,High density lipoprotein cholesterol levels,High density lipoprotein cholesterol levels
INITIAL SAMPLE SIZE,"19,223 British ancestry individuals from 6863 ...","76,627 European ancestry individuals, 7,795 Hi...","76,627 European ancestry individuals, 7,795 Hi...","94,595 European ancestry individuals","34,930 East Asian ancestry individuals, 187,16...","76,627 European ancestry individuals, 7,795 Hi...","76,627 European ancestry individuals, 7,795 Hi...","76,627 European ancestry individuals, 7,795 Hi...","76,627 European ancestry individual